# U.S. Census Geocoder
- This notebook reads in address data and geocodes it using the U.S. Census Geocoder (https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form).  


- The census benchmark is set to **Public_AR_Current**.


- Incoming data should only include the following fields.
    - address
    - city
    - state (cd)
    - zip

- The notebook will create a single dataframe called **results_df** that can be exported.

In [ ]:
import pandas as pd

### Remove the cell below when using in alteryx

In [ ]:
# this should not be needed when reading data from alteryx
path = 'C:/Census Dataset/'

### Edit the below cell when using in Alteryx

In [ ]:
# pd.read_csv(path + 'Census Geo_uniq addess.csv') should be changed to read the data in from alteryx. 
df = pd.read_csv(path + 'Census Geo_uniq addess.csv')

Drop records with any part of the address missing.  Addresses missing columns will not have a match.  This will reduce processing time.

In [ ]:
df.dropna(inplace=True)

Format zip code to prevent errors

In [ ]:
df. = df.zip.astype('int32')

### Create Batch IDs

In [ ]:
def create_batch_id(data):
    batch_sz = 9999
    batch_id=0
    record_ct=0
    for index, row in data.iterrows():
        if record_ct == batch_sz:
            record_ct = 0 # reset record count when batch size is reached
            
        if record_ct == 0:
            batch_id = batch_id + 1  #add 1 to batch ID every time record count is reset
            
        record_ct = record_ct + 1
        data.at[index,'batch']=batch_id
    return data

In [ ]:
batch_df = create_batch_id(df)

## API Time

The script below allows us to query the US Census website and geocode addresses.  

In [ ]:
import requests

def get_geo(data):
    c_file = data.to_csv(header = False)
    url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
    payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'} 
    files = {'addressFile': ('Addresses.csv', c_file, 'text/csv')}
    r = requests.post(url, files=files, data = payload)
    return(r.text)

Send addresses in batches through the U.S. Census Geocoder and append results to dataframe.

In [ ]:
import io

#create empty dataframe to hold addresses after processing.
results_df = pd.DataFrame({'record_id':[], 'address':[], 'match/no_match':[], 'match_type':[], 
                           'geo_address':[], 'lat_long':[], 'tiger_line_id':[], 
                           'side':[], 'state_id':[], 'county_id':[], 'tract_id':[], 'block_id':[]})

#identify the number of batches to be processed and use number in loop below.
batch_ct = max(batch_df['batch'])

# must define column names to prevent read_csv from thinking there are less columns than there truly are.
names = ['record_id', 'address', 'match/no_match', 'match_type', 'geo_address', 'lat_long', 'tiger_line_id', 'side', 'state_id', 'county_id', 'tract_id','block_id']

#set counter for loop
i=1

while i <= batch_ct:
    c_file = batch_df[batch_df['batch']==batch_ct]
    c_file = c_file.drop('batch', axis=1)
    geo_file = get_geo(c_file)
    results_df = results_df.append(pd.read_csv(io.StringIO(geo_file), names = names))
    i = i + 1